#  Introduction

In this tutorial, we will train logistic/softmax regression model using TensorFlow, to predict if the input is neutral face or smiling face.
About the dataset, I collected images from Google Images and cropped the mouth part is manually put them into two separate folders. 
This is developed in Python 2.7.10 and Tensorflow 1.2.1

In [15]:
import cv2
import glob,os
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
print ("tensorflow version",tf.__version__)
%matplotlib inline

tensorflow version 1.13.1


# Loading data and resize 

In [36]:
# reading images from the data folder
# label, smile = 1, neutral = 0
(img_width, img_height) = (45, 25)
def load_images_from_folder(folder):
    (images, lables, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(folder):
        print (subdirs, dirs)
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(folder, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                lable = id
                img = cv2.imread(path, 0) 
                img = cv2.resize(img,(img_width, img_height)) 
                images.append(img)
                lables.append(int(lable))
            id += 1
        print(names)
        print(lables)
        return images, lables, names


In [37]:
X,Y,classes = load_images_from_folder("/home/ayushi/Real-time-Smile-Recognition/data")

(X,Y) = [np.array(lis) for lis in [X, Y]]
Y = pd.get_dummies(Y) #converting labels to one-hot, Used Pandas for it. 

/home/ayushi/Real-time-Smile-Recognition/data ['smile', 'neutral']
{0: 'smile', 1: 'neutral'}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Dimension of our data

In [38]:
print ("Height of the image:  " + str(X.shape[1]))
print ("Width of the image:   "  + str(X.shape[2]))
print ("Total number of data: "+ str(len(X)))

Height of the image:  25
Width of the image:   45
Total number of data: 107


# Splitting the data using sklearn 

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)

# Size of data, Train set and Test set

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)
print ("lenght of train data: "+str(len(X_train))) 
print ("lenght of test data:  "+str(len(X_test)))

lenght of train data: 71
lenght of test data:  36


In [41]:
train_set_x_flatten = X_train.reshape(X_train.shape[0],-1)
test_set_x_flatten = X_test.reshape(X_test.shape[0],-1)
print ("train_set_x_flatten: "+str(train_set_x_flatten.shape))
print ("test_set_x_flatten : "+str(test_set_x_flatten.shape))

print (train_set_x_flatten)

train_set_x_flatten: (71, 1125)
test_set_x_flatten : (36, 1125)
[[159 157 166 ... 145 143 142]
 [146 149 140 ... 149 145 146]
 [204 208 210 ... 206 207 194]
 ...
 [162 164 174 ... 162 161 158]
 [170 181 179 ... 154 159 155]
 [112 116 113 ... 136 132 132]]


In [42]:
#Normalizing data, conveting all pixel value in range between 0-1
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.
print ("length of train set: " + str(len(train_set_x)))
print ("length of test set: " + str(len(test_set_x)))
print ("shape of train set: "+ str(train_set_x.shape))
print ("shape of test set: "+ str(test_set_x.shape))


length of train set: 71
length of test set: 36
shape of train set: (71, 1125)
shape of test set: (36, 1125)


# Building the softmax regression using tensorflow

In [43]:
import tensorflow as tf

In [44]:
x = tf.placeholder(tf.float32, [None, 1125])
W = tf.Variable(tf.zeros([1125, 2]), dtype=tf.float32, name="w1")
b = tf.Variable(tf.zeros([2]),  dtype=tf.float32, name="bias")
y = tf.nn.softmax(tf.matmul(x, W) + b, name="first_operation")
y_ = tf.placeholder(tf.float32, [None, 2], name = "final_output")
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [80]:
learning_rate = 0.1
print ("training with learning rate: " + str(learning_rate))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(15000):
        trainStep,loss = sess.run([train_step, cross_entropy], feed_dict={x: train_set_x, y_: y_train})
        if step%1000==0:
            print ("losses after per 1000 iteration: ",loss)

    save_path = saver.save(sess, "./model/emotion_model")

    print("Model saved in file: %s" % save_path)

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("accuracy with learning rate: " ,str(learning_rate), sess.run(accuracy, feed_dict={x:test_set_x , y_: y_test}))

training with learning rate: 0.1
losses after per 1000 iteration:  0.6931472
losses after per 1000 iteration:  0.016667362
losses after per 1000 iteration:  0.009934486
losses after per 1000 iteration:  0.007109392
losses after per 1000 iteration:  0.0055379937
losses after per 1000 iteration:  0.0045358283
losses after per 1000 iteration:  0.0038408486
losses after per 1000 iteration:  0.003330561
losses after per 1000 iteration:  0.0029399844
losses after per 1000 iteration:  0.0026314124
losses after per 1000 iteration:  0.0023814947
losses after per 1000 iteration:  0.0021749528
losses after per 1000 iteration:  0.002001402
losses after per 1000 iteration:  0.0018535227
losses after per 1000 iteration:  0.0017260044
Model saved in file: ./model/emotion_model
accuracy with learning rate:  0.1 0.5833333


# prediction 

In [83]:
saver = tf.train.Saver()
with tf.Session() as sess:
    #saver.restore(sess, "./model/emotion_model")
    print("w1:", sess.run(W))
    #print("bias:", sess.run(b))
    img = cv2.imread("/home/ayushi/Real-time-Smile-Recognition/test/test1.jpg", 0) 
    img = cv2.resize(img,(img_width, img_height)) # Resizing all the images
    image=[]
    image.append(img)
    image_test = np.array(image)
    test_image_flatten = image_test.reshape(image_test.shape[0],-1)
    test_image_normalized = test_image_flatten/255.
    #print (test_image_normalized.shape)
    x_ = tf.cast(test_image_normalized, tf.float32)
    y = tf.nn.softmax(tf.matmul(x_, W) + b)
    #print (sess.run(y))
    indx = sess.run(tf.argmax(y,1))
    if indx==0:
        print ("smile")
    elif indx==1:
        print ("neutral")

INFO:tensorflow:Restoring parameters from ./model/emotion_model
neutral
